In [1]:
import numpy as np
import pandas as pd
import sympy

import FormulaLab as fl

### FormulaLab* allows you to:

#### 1. Search for formula(s) in two ways:

    1. By direct search.
            FormulaLab.FormulaSearch.get(function, [variables,])
  
    2. By going through the formulas database and algebraically solve for the desired formula 
            FormulaLab.FormulaSearch.find(function, variable)
            
#### 2. Convert symbolic (text) formulas into python functions, where you can subtitute numbers and evaluate.
            FormulaLab.FormulaSearch.function(formula)
            FormulaLab.FormulaSearch.get(function, [variables,], function=True)
            
#### 3. Retrieve stored formula from a database by its ID number, and use it anywhere in your code with any form.

    FormulaLab gives you the comfort to store all of your formulas of different projects in one place (database) and 
    retrieve them with any form by caling them by their id, or by searching as above. For example, Say you stored the 
    following formula "F = m * a", with id number 1. And your code uses this formula in three different places/projects 
    with different forms: "f=m*a", "m=f/a", and "a=F/m". Instead of writing them explicitly three times in your code, you 
    just call them from the database as:
            f = FormulaLab.FormulaSearch.get(function='f', id=1, function=True)  # first place
            m = FormulaLab.FormulaSearch.get(function='m', id=1, function=True)  # second place
            a = FormulaLab.FormulaSearch.get(function='a', id=1, function=True)  # third place
            
            
\* FormulaLab is bult upon [sympy](https://www.sympy.org/en/index.html)

# Step 1: Importing Formulas from a Database

### 1. Data from list, dict, tuple, or set
    Yor data could be stored in list, dict, tuple, or set. However, it is recomeneded that you store your data in a file 
    (eg., excel file) and then import it by pandas as you will see soon.

In [2]:
data_list = ['f=m*a','a=v/t']
data_dict = {'1':'f=m*a','2':'a=v/t'}
data_dict_2 = {'ID':[1, 2], 'Formula':['f=m*a','a=v/t']}
data_tuple = ('f=m*a','a=v/t')
data_set = {'f=m*a','a=v/t'}

data_list

['f=m*a', 'a=v/t']

### 2. Data from external Database, should be imported as a DataFrame
    This is the recommended way of storing and loading formulas, and pandas is one of the best python packages that 
    can handle this

In [14]:
df = pd.read_csv('Quick Database.csv')
df

,ID,Formula
0,1,d = v * t
1,2,a = v / t
2,3,f = m * a
3,5,p = m * v


# Step 2: Initiate the formula search engine

In [15]:
phyfos = fl.FormulaSearch(data=data_list) 
phyfos.data
# The new "Args" column is automatically generated to help speed the search algorithm.  

,ID,Formula,Args
0,1,f=m*a,"[f, m, a]"
1,2,a=v/t,"[a, v, t]"


In [16]:
phyfos = fl.FormulaSearch(data=data_dict) 
phyfos.data

,ID,Formula,Args
0,1,f=m*a,"[f, m, a]"
1,2,a=v/t,"[a, v, t]"


In [17]:
phyfos = fl.FormulaSearch(data=df, formula_col='Formula', id_col='ID') 
#formula_col and id_col values should mathch the formulas database
phyfos.data

,ID,Formula,Args
0,1,d = v * t,"[d, v, t]"
1,2,a = v / t,"[a, v, t]"
2,3,f = m * a,"[f, m, a]"
3,5,p = m * v,"[p, m, v]"


# Step 3: Search for formula(s)

### 1-  Direct search

In [18]:
d = phyfos.get('d')
d

[[t*v]]

In [20]:
v = phyfos.get('v')
v

[[d/t], [a*t], [p/m]]

In [22]:
v_as_a_function_of_a = phyfos.get('v', 'a')
v_as_a_function_of_a

[[a*t]]

In [26]:
a_as_a_function_of_v = phyfos.get('a', 'v')
a_as_a_function_of_v

[[v/t]]

In [24]:
v_as_a_function_of_t_d = phyfos.get('v', ['t','d'])
v_as_a_function_of_t_d

[[d/t]]

### Search by formula ID number
    Each formula has a fixed id number that should not be changed. It is recomeneded to use this method when you know what formula to use in your code. It is faster than direct search, and the later code give you different result when the database is expanded or changed.

In [28]:
f = phyfos.get('f', id=3)
f

[a*m]

In [30]:
# Say you want "m" to be a function, you do not have to rewrite your formula again!
m = phyfos.get('m', id=3)
m

[f/a]

In [29]:
# Also for 'a'. This is very helpful when there is a mistake in the formula! You only 
#change it once in your database.
a = phyfos.get('a', id=3)
a

[f/m]

### 2- Deriving formulas, by connection to variables through algabric substitutions
    This is the most interesting part! It uses the full power of FormulaLab. FormulaLab uses some algorithms to find all possible paths (intermediate formulas/variables) between a function and a variable, and derive a new formula that connects the two of them and is not in the database. FormulaLab can do algebraic manibulations even integrals and derivatives!

In [9]:
Ef = phyfos.find('Electric_field', 'Acceleration')
Ef

[Acceleration*mass/charge,
 Electric_potential/(Acceleration*time**2),
 Acceleration*Electric_potential/Velocity**2]

In [10]:
sympy.pretty_print(Ef[0])

Acceleration⋅mass
─────────────────
      charge     


## 3- Converting formula expresion to a python function

In [11]:
Ef_func = phyfos.function(Ef[0])
Ef_func(Acceleration=10, mass=5, charge=2)

25.0

In [12]:
Weight_func = phyfos.get('Weight', function=True)
Weight_func(Gravety=9.8, mass=80)

784.0